In [3]:
import requests, time
from bs4 import BeautifulSoup as bs
import pandas as pd

In [4]:
vehicle_urls = []


def fetch_page_and_convert_to_bs(url):

    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    
    # Delay 5 seconds before returning the soup object to prevent Python from sending too many requests in a rapid succession
    time.sleep(5)

    return soup


def get_vehicle_urls():
    
    counter = 1
    iterate = True
    pages = 0
        
    # Storing the base url the data will be fetched from
    base_url = 'https://www.cars.com/for-sale/searchresults.action/?dealerType=all&perPage=50&rd=10&searchSource=PAGINATION&sort=relevance&page='

    # Running a loop to determine how many pages with vehicle content exist
    while (iterate):
        
        """ 
            Creating a dynamic url that will update the page number in each pass of the loop
            until there is no vehicle content to be found
        """

        url = f'{base_url}{counter}'
        
        # Instantiating a beautiful soup object to store the html
        soup = fetch_page_and_convert_to_bs(url)        
        
        # Identify if any vehicles exist on the page
        has_vehicle = len(soup.find('a', class_ = 'shop-srp-listings__listing'))

        if (has_vehicle and pages < 3): # The "3" in the conditional to prevent the return of the entire cars.com list.
            pages += 1
            counter += 1
            
            # Find all the vehicles that exist on the page
            vehicles = soup.find_all('a', class_ = 'shop-srp-listings__listing')
            
            for vehicle in vehicles:
                vehicle_urls.append(vehicle['href'])
            
        else:
            iterate = False


In [5]:
get_vehicle_urls()
url=vehicle_urls
print(url)

['/vehicledetail/detail/774982419/overview/', '/vehicledetail/detail/780950135/overview/', '/vehicledetail/detail/779718086/overview/', '/vehicledetail/detail/780954200/overview/', '/vehicledetail/detail/778983999/overview/', '/vehicledetail/detail/778854490/overview/', '/vehicledetail/detail/773416275/overview/', '/vehicledetail/detail/778295222/overview/', '/vehicledetail/detail/779965041/overview/', '/vehicledetail/detail/766287718/overview/', '/vehicledetail/detail/762540854/overview/', '/vehicledetail/detail/776310800/overview/', '/vehicledetail/detail/766658922/overview/', '/vehicledetail/detail/767775554/overview/', '/vehicledetail/detail/782078156/overview/', '/vehicledetail/detail/772675368/overview/', '/vehicledetail/detail/777228689/overview/', '/vehicledetail/detail/769623554/overview/', '/vehicledetail/detail/764649931/overview/', '/vehicledetail/detail/782588730/overview/', '/vehicledetail/detail/777810435/overview/', '/vehicledetail/detail/772073658/overview/', '/vehicle

In [6]:
new_url=[]
for i in url:
    complete_list=(f'https://www.cars.com'+i)
    new_url.append(complete_list)
print(len(new_url))


150


In [7]:
# Now that the urls for each individual vehicle have been found use the fetch_page_and_convert_to_bs() function...
# each pass of the loop should scrape the pertinent data points from the page
# each scrape should store the data points as a dictionary

final_data=[]
counter = 0
vehicles = []

def get_car_info(url):
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')
        # Delay 5 seconds before returning the soup object to prevent Python from sending too many requests in a rapid succession    
        time.sleep(3)

        return soup  
    
    
for url in new_url:
    soup=get_car_info(url)
    
    #Confirms that the listing includes 
    try:
        year_make_model = soup.find('p', class_ = 'mmy-link').a.text
        year_make_model_list = year_make_model.split()
        list_len=len(year_make_model_list)
        #Evaluates that all required data elements have been supplied but checking the greatest index element.
        mileage = soup.find_all('li', class_ = 'vdp-details-basics__item')[10].span.text

    except:
        list_len=0

    
    if list_len==3:

        source_id = 1
        year=year_make_model_list[0]
        make=year_make_model_list[1]
        model=year_make_model_list[2]

        condition = soup.find('h1', class_ = 'vehicle-info__stock-type').text
        condition = condition.title()
        price = soup.find('span', class_='vehicle-info__price-display').text
        vin = soup.find_all('li', class_ = 'vdp-details-basics__item')[9].span.text
        mileage = soup.find_all('li', class_ = 'vdp-details-basics__item')[10].span.text
        transmission = soup.find_all('li', class_ = 'vdp-details-basics__item')[7].span.text
        engine = soup.find_all('li', class_ = 'vdp-details-basics__item')[8].span.text
        drive_type = soup.find_all('li', class_ = 'vdp-details-basics__item')[6].span.text
        fuel_type = soup.find_all('li', class_ = 'vdp-details-basics__item')[0].span.text
        exterior_color = soup.find_all('li', class_ = 'vdp-details-basics__item')[1].span.text
        interior_color = soup.find_all('li', class_ = 'vdp-details-basics__item')[3].span.text
        seller_name = soup.find('h3', class_ = 'vdp-dealer-info__title').text
        seller_address = soup.find('p', class_ = 'vdp-dealer-location__address').text
        
        #Parsing address field to extract zip code
        address_parts = seller_address.split()
        where_to_pull=len(address_parts)
        zip_code = address_parts[where_to_pull-1]
        
        vehicle = {
        'source_id' : source_id,
        'condition' : condition,
        'year': year,
        'make' : make,
        'model' : model,
        'price' : price,
        'drive_type' : drive_type,
        'engine' : engine,
        'seller_name' : seller_name,
        'fuel_type' : fuel_type,
        'exterior_color' : exterior_color,
        'interior_color' : interior_color,
        'mileage' : mileage,
        'transmission' : transmission,
        'vin' : vin,
        'url' : url

        }
        
        vehicles.append(vehicle)

In [8]:
print(vehicles)

[{'source_id': 1, 'condition': 'New', 'year': '2019', 'make': 'Toyota', 'model': 'Tundra', 'price': '$41,419', 'drive_type': ' 4WD', 'engine': ' 5.7L V8 32V MPFI DOHC', 'seller_name': 'Texas Toyota of Grapevine', 'fuel_type': ' Gasoline', 'exterior_color': ' Midnight Black', 'interior_color': ' Graphite', 'mileage': ' 2', 'transmission': ' 6-Speed Automatic', 'vin': ' 5TFDY5F16KX849034', 'url': 'https://www.cars.com/vehicledetail/detail/773416275/overview/'}, {'source_id': 1, 'condition': 'New', 'year': '2019', 'make': 'Nissan', 'model': 'Armada', 'price': '$46,300', 'drive_type': ' RWD', 'engine': ' 5.6L V8 32V GDI DOHC', 'seller_name': 'Texas Nissan of Grapevine', 'fuel_type': ' Gasoline', 'exterior_color': ' Hermosa Blue', 'interior_color': ' Charcoal', 'mileage': ' 5', 'transmission': ' 7-Speed Automatic', 'vin': ' JN8AY2ND6K9093923', 'url': 'https://www.cars.com/vehicledetail/detail/779965041/overview/'}, {'source_id': 1, 'condition': 'New', 'year': '2019', 'make': 'Nissan', 'mode

In [9]:
df = pd.DataFrame(vehicles)

In [10]:
df.head()

,condition,drive_type,engine,exterior_color,fuel_type,interior_color,make,mileage,model,price,seller_name,source_id,transmission,url,vin,year
0,New,4WD,5.7L V8 32V MPFI DOHC,Midnight Black,Gasoline,Graphite,Toyota,2,Tundra,"$41,419",Texas Toyota of Grapevine,1,6-Speed Automatic,https://www.cars.com/vehicledetail/detail/7734...,5TFDY5F16KX849034,2019
1,New,RWD,5.6L V8 32V GDI DOHC,Hermosa Blue,Gasoline,Charcoal,Nissan,5,Armada,"$46,300",Texas Nissan of Grapevine,1,7-Speed Automatic,https://www.cars.com/vehicledetail/detail/7799...,JN8AY2ND6K9093923,2019
2,New,FWD,3.5L V6 24V MPFI DOHC,Magnetic Black,Gasoline,Graphite,Nissan,5,Murano,"$32,500",Texas Nissan of Grapevine,1,Automatic CVT,https://www.cars.com/vehicledetail/detail/7662...,5N1AZ2MJ4KN120104,2019
3,New,RWD,2.7L V6 24V PDI DOHC Twin Turbo,Ingot Silver,Gasoline,Gray,Ford,7,F-150,"$30,490",Grapevine Ford Lincoln,1,10-Speed Automatic,https://www.cars.com/vehicledetail/detail/7625...,1FTEW1CP4KKC47533,2019
4,New,FWD,Intercooled Turbo Regular Unleaded I-4 1.5 L/91,Lunar Silver Metallic,Gasoline,Gray,Honda,10,Accord,"$26,288",David McDavid Honda Irving,1,1-Speed CVT w/OD,https://www.cars.com/vehicledetail/detail/7696...,1HGCV1F40KA075701,2019
